# 3.6 

The file m sp500ret 3mtcm.txt contains three columns. The second column gives the monthly returns of the S&P 500 index from January 1994
to December 2006. The third column gives the monthly rates of the
3-month U. S. Treasury bill in the secondary market, which is obtained
from the Federal Reserve Bank of St. Louis and used as the risk-free asset
here. Consider the ten monthly returns in the file m ret 10stocks.txt.

**Pre data cleaning**

In [96]:
import statsmodels.api as sm
from scipy.stats import f, t
import pandas as pd
import numpy as np

In [85]:
# Load and format data
market_df = pd.read_csv('m_sp500ret_3mtcm.csv')
stocks_log_df = pd.read_csv('ret-10-stocks.csv')
stocks_log_df['Date'] = pd.to_datetime(stocks_log_df['Date'])

In [86]:
stocks_names = stocks_log_df.columns[1:]

# Create empty dataset to store normal returns
stocks_ret = pd.DataFrame()

# stocks_ret = stocks_log_df[stocks_names].copy()
# Compute normal returns for each ticker
for stock in stocks_names:
    stocks_ret[stock] = stocks_log_df[stock].apply(lambda x: np.exp(x) - 1)

In [90]:
# Merge the two datasets
market_stocks_df = pd.concat([market_df[['sp500', '3mTCM']], stocks_ret], axis=1)

num_obs = len(market_stocks_df)
market_stocks_df.index = pd.date_range(start='1994-01-01', periods=num_obs, freq='MS')

# Cast treasury bill rates to decimal instead of percentage
market_stocks_df['3mTCM'] /= 100 

# Market excess return (MER)
market_stocks_df['MER'] = market_stocks_df['sp500'] - market_stocks_df['3mTCM']

market_stocks_df.head(3)

,sp500,3mTCM,AAPL,ADBE,ADP,AMD,DELL,GTW,HP,IBM,MSFT,ORCL,MER
1994-01-01,0.013890,0.0298,0.050129,0.144829,-0.022636,0.064293,-0.012510,0.093338,-0.004323,0.000000,0.023759,0.050335,-0.015910
1994-02-01,-0.013248,0.0325,0.050061,-0.014823,-0.009462,0.020900,0.061397,-0.002164,-0.014283,-0.026493,-0.013451,0.010849,-0.045748
1994-03-01,-0.020336,0.0350,-0.039941,-0.075894,0.000873,0.176181,0.000000,-0.068942,-0.007917,0.014270,0.011702,-0.010732,-0.055336


In [92]:
# Calculate stock excess returns
excess_ret_names = []

for name in stocks_names:
    excess_name = f'{name}_e'
    market_stocks_df[excess_name] = market_stocks_df[name] - market_stocks_df['3mTCM']
    excess_ret_names.append(excess_name)

In [103]:
market_stocks_df.head(3)

,sp500,3mTCM,AAPL,ADBE,ADP,AMD,DELL,GTW,HP,IBM,...,AAPL_e,ADBE_e,ADP_e,AMD_e,DELL_e,GTW_e,HP_e,IBM_e,MSFT_e,ORCL_e
1994-01-01,0.013890,0.0298,0.050129,0.144829,-0.022636,0.064293,-0.012510,0.093338,-0.004323,0.000000,...,0.020329,0.115029,-0.052436,0.034493,-0.042310,0.063538,-0.034123,-0.029800,-0.006041,0.020535
1994-02-01,-0.013248,0.0325,0.050061,-0.014823,-0.009462,0.020900,0.061397,-0.002164,-0.014283,-0.026493,...,0.017561,-0.047323,-0.041962,-0.011600,0.028897,-0.034664,-0.046783,-0.058993,-0.045951,-0.021651
1994-03-01,-0.020336,0.0350,-0.039941,-0.075894,0.000873,0.176181,0.000000,-0.068942,-0.007917,0.014270,...,-0.074941,-0.110894,-0.034127,0.141181,-0.035000,-0.103942,-0.042917,-0.020730,-0.023298,-0.045732


(a). Fit CAPM to the ten stocks. Give point estimates and 95% confidence intervals of $\alpha$, $\beta$, the Sharpe index, and the Treynor index (Hint: Use the delta method for the Sharpe and Treynor indices.)

In [100]:
# Constants

ALPHA_LEVEL = 0.05   # 95% confidence interval

In [98]:
def fit_capm(df, stock_e_names, MER):
    '''
    Fit CAPM. Give point estimates and 95% CIs for alpha, beta.
    Sharpe ratio, and Treynor index.
    '''

    model_list = []
    alphas_list = []

    for stock_e_name in stock_e_names:
        print(f'Analysis for: {stock_e_name}')
        Y = df[stock_e_name]
        X = sm.add_constant(MER)
        n = len(Y)

        # Fit OLS model
        model = sm.OLS(Y, X).fit()
        model_list.append(model)

        # Alpha, Beta estimates & CIs
        alpha_hat = model.params['const']
        beta_hat = model.params['MER']
        alphas_list.append(alpha_hat)
        alpha_ci = model.conf_int(ALPHA_LEVEL).loc['const'].values
        beta_ci = model.conf_int(ALPHA_LEVEL).loc['MER'].values

        # Sharpe Ratio
        mu_e_hat = Y.mean()
        sigma_e_hat = Y.std(ddof=1)
        sharpe_hat = mu_e_hat / sigma_e_hat

        # Sharpe CI (Delta Method, Var(S) ~ (1 + 0.5*S^2)/n)
        se_sharpe_delta = np.sqrt((1 + 0.5 * sharpe_hat**2) / n)
        z_crit = t.ppf(1 - ALPHA_LEVEL / 2, n - 1)
        sharpe_ci = [sharpe_hat - z_crit * se_sharpe_delta, sharpe_hat + z_crit * se_sharpe_delta]

        # Treynor Ratio
        mu_EMR_hat = MER.mean()
        treynor_hat = mu_e_hat + alpha_hat / beta_hat

        # Treynor CI (Delta Method)
        grad_g = np.array([1 / beta_hat, - alpha_hat / (beta_hat**2)])
        cov_ab = model.cov_params().loc[['const', 'MER'], ['const', 'MER']].values
        var_treynor_delta = grad_g.T @ cov_ab @ grad_g
        se_treynor_delta = np.sqrt(var_treynor_delta)
        treynor_ci = [treynor_hat - z_crit * se_treynor_delta, treynor_hat + z_crit * se_treynor_delta]

        print(f"  Alpha   : {alpha_hat:9.6f}, CI: [{alpha_ci[0]:9.6f}, {alpha_ci[1]:9.6f}]")
        print(f"  Beta    : {beta_hat:9.6f}, CI: [{beta_ci[0]:9.6f}, {beta_ci[1]:9.6f}]")
        print(f"  Sharpe  : {sharpe_hat:9.6f}, CI: [{sharpe_ci[0]:9.6f}, {sharpe_ci[1]:9.6f}]")
        print(f"  Treynor : {treynor_hat:9.6f}, CI: [{treynor_ci[0]:9.6f}, {treynor_ci[1]:9.6f}]")

    return model_list, alphas_list

In [102]:
results_a = fit_capm(market_stocks_df, excess_ret_names, market_stocks_df['MER'])

Analysis for: AAPL_e
  Alpha   :  0.021970, CI: [ 0.004322,  0.039617]
  Beta    :  1.447559, CI: [ 1.032207,  1.862911]
  Sharpe  : -0.425037, CI: [-0.590183, -0.259891]
  Treynor : -0.014363, CI: [-0.023231, -0.005494]
Analysis for: ADBE_e
  Alpha   :  0.018576, CI: [ 0.000712,  0.036439]
  Beta    :  1.329802, CI: [ 0.909368,  1.750235]
  Sharpe  : -0.417337, CI: [-0.582238, -0.252437]
  Treynor : -0.014775, CI: [-0.024804, -0.004746]
Analysis for: ADP_e
  Alpha   : -0.004426, CI: [-0.011235,  0.002383]
  Beta    :  0.841128, CI: [ 0.680876,  1.001381]
  Sharpe  : -1.124404, CI: [-1.326459, -0.922350]
  Treynor : -0.039618, CI: [-0.048569, -0.030667]
Analysis for: AMD_e
  Alpha   :  0.034373, CI: [ 0.012524,  0.056222]
  Beta    :  1.888061, CI: [ 1.373819,  2.402302]
  Sharpe  : -0.376489, CI: [-0.540155, -0.212823]
  Treynor : -0.014606, CI: [-0.022504, -0.006708]
Analysis for: DELL_e
  Alpha   :  0.021826, CI: [ 0.006433,  0.037220]
  Beta    :  1.314299, CI: [ 0.951991,  1.67660

Use the bootstrap procedure in Section 3.5 to estimate the standard errors of the point estimates of $\alpha$, $\beta$, and the Sharpe and Treynor indices.

In [ ]:
def bootstrap(df, stock_e_names, MER):
    '''
    Bootstrap procedure to estimate standard errors of 
    alpha, beta, Sharpe, and Treynor.
    '''
    
    for stock_e_name in stock_e_names:
        print(f"\nAnalysis for: {stock_e_name}")
        
        Y = df[stock_e_name]
        n = len(Y)

        boot_alphas = []
        boot_betas = []
        boot_sharpes = []
        boot_treynors = []
        
        Y_np = Y.values
        MER_np = MER.values
        
        for _ in range(NUM_BOOTSTRAP):
            indices = np.random.choice(range(n), n, replace=True)
            Y_boot = Y_np[indices]
            r_M_e_boot_series = r_M_e_np[indices]
            X_boot = sm.add_constant(r_M_e_boot_series)
            
            try:
                model_boot = sm.OLS(Y_boot, X_boot).fit()
                a_boot = model_boot.params[0]
                b_boot = model_boot.params[1]
                s_boot = Y_boot.mean() / Y_boot.std(ddof=1)
                t_boot = a_boot / b_boot + r_M_e_boot_series.mean()
                
                if np.isfinite([a_boot, b_boot, s_boot, t_boot]).all():
                    boot_alphas.append(a_boot)
                    boot_betas.append(b_boot)
                    boot_sharpes.append(s_boot)
                    boot_treynors.append(t_boot)
            except Exception:
                continue 

        print(f"  SE(Alpha)  : {np.std(boot_alphas, ddof=1):9.6f}")
        print(f"  SE(Beta)   : {np.std(boot_betas, ddof=1):9.6f}")
        print(f"  SE(Sharpe) : {np.std(boot_sharpes, ddof=1):9.6f}")
        print(f"  SE(Treynor): {np.std(boot_treynors, ddof=1):9.6f}")